In [1]:
# Costuma ser exigida a instalação destas bibliotecas no primeiro acesso ao Pysus
!apt-get update  
!apt-get install libffi-dev  # Install libffi-dev
!pip install --upgrade pip  # Ensure pip is up to date
!pip install pysus # Exa
!pip install gdown
!pip install pylance

Reading package lists... Done
E: List directory /var/lib/apt/lists/partial is missing. - Acquire (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [2]:
pip install pylance

Note: you may need to restart the kernel to use updated packages.


In [1]:
# importar bibliotecas para carga dos dados
import pysus
import os
import sidrapy
import requests # para carga de endereços web - api
from pysus import SIM
from pysus.online_data.SIM import download
from pysus.preprocessing.decoders import translate_variables_SIM
from pysus.preprocessing.SIM import group_and_count, redistribute_missing, redistribute_cid_chapter
from pysus.online_data.SIM import get_CID9_table, get_CID10_table, get_municipios, get_ocupations
from ftplib import FTP
from pathlib import Path  

In [6]:
# importar biblioteca básicas para manipulação de dados
import pandas as pd
import numpy as np
import os
import sidrapy
import requests # para carga de endereços web - api
import gdown
import openpyxl
from tabulate import tabulate

In [12]:

dobr2010 = sim.get_files("CID10", uf=["BR"], year=[2010])
dobr2010

NameError: name 'sim' is not defined

In [9]:
translate_variables_SIM

<function pysus.preprocessing.decoders.translate_variables_SIM(dataframe: pandas.core.frame.DataFrame, age_unit: str = 'Y', age_classes=None, classify_args: dict = {}, classify_cid10_chapters=False, geocode_dv=True, nan_marker=None, category_columns=True)>

In [27]:
dobr_clean = pd.read_parquet('files_temp/dobr_clean.parquet')
dobr_clean_19a22 = pd.read_parquet('files_temp/dobr_clean_19a22.parquet')
AvoidMort_v4_dobr_10_19a22 = pd.read_parquet('files_temp/AvoidMort_v4_dobr_10_19a22.parquet')
AvoidMort_v5_dobr_10_19a22 = pd.read_parquet('files_temp/AvoidMort_v5_dobr_10_19a22.parquet')

In [29]:
for tab in [dobr_clean, dobr_clean_19a22, AvoidMort_v4_dobr_10_19a22, AvoidMort_v5_dobr_10_19a22]:
    print(f'Registro da tabela : {tab.shape[0]}')
    if 'CONTADOR' in tab.columns:
        print(f'Contador de óbitos : {tab.CONTADOR.sum()}')
    print(tabulate(tab.head(2), headers='keys', tablefmt='psql', maxcolwidths=50))

Registro da tabela : 872731
Contador de óbitos : 876602
+----+------------+-------------+---------+------------+-------------+
|    | CAUSABAS   | CODMUNRES   |   IDADE |   CONTADOR | ANO_OBITO   |
|----+------------+-------------+---------+------------+-------------|
|  0 | A047       | _355030     |      69 |          1 | _2022       |
|  1 | A09        | _411050     |      52 |          1 | _2022       |
+----+------------+-------------+---------+------------+-------------+
Registro da tabela : 3667555
Contador de óbitos : 3724693
+----+------------+-------------+---------+------------+-------------+
|    | CAUSABAS   | CODMUNRES   |   IDADE |   CONTADOR | ANO_OBITO   |
|----+------------+-------------+---------+------------+-------------|
|  0 | A049       | _355080     |      52 |          1 | _2019       |
|  1 | A09        | _320520     |      67 |          1 | _2019       |
+----+------------+-------------+---------+------------+-------------+
Registro da tabela : 4446922
+----

In [32]:
# mostrar dados gerais da tabela após transformações
tab = AvoidMort_v4_dobr_10_19a22
print(f'Registro da tabela : {tab.shape[0]}')
# print(f'Contador de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql', maxcolwidths=50))

Registro da tabela : 4446922
+----+------------+-------------+-------------+
|    | CAUSABAS   | CODMUNRES   | ANO_OBITO   |
|----+------------+-------------+-------------|
|  0 | I219       | _292740     | _2019       |
|  1 | I10        | _355100     | _2019       |
|  2 | X700       | _500630     | _2019       |
|  3 | C539       | _500500     | _2019       |
|  4 | V220       | _500627     | _2019       |
+----+------------+-------------+-------------+


In [33]:
# mostrar dados gerais da tabela após transformações
tab = AvoidMort_v5_dobr_10_19a22
print(f'Registro da tabela : {tab.shape[0]}')
# print(f'Contador de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql', maxcolwidths=50))

Registro da tabela : 4446922
+----+------------+-------------+--------+-----------+-------------+
|    | CAUSABAS   | CODMUNRES   |   SEXO |   RACACOR | ANO_OBITO   |
|----+------------+-------------+--------+-----------+-------------|
|  0 | I219       | _292740     |      1 |         2 | _2019       |
|  1 | I10        | _355100     |      2 |         1 | _2019       |
|  2 | X700       | _500630     |      1 |         1 | _2019       |
|  3 | C539       | _500500     |      2 |         4 | _2019       |
|  4 | V220       | _500627     |      1 |         1 | _2019       |
+----+------------+-------------+--------+-----------+-------------+


In [30]:
dobr_clean.ANO_OBITO.unique()


array(['_2022'], dtype=object)